# **Chapter 6: Precognition (Thinking Step by Step)**

---
**Lesson:**

If someone woke you up and immediately started asking you several complicated questions that you had to respond to right away, how would you do? Probably not as good as if you were given time to think through your answer first. 
Guess what? Claude is the same way.
Giving Claude time to think step by step sometimes makes Claude more accurate, particularly for complex tasks.

---

First we will setup our dependencies

In [3]:
%%capture
#Install dependencies
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

%pip install --quiet langchain==0.0.304

#Import libraries, and set up Bedrock client
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

#Set up model inference modifiers, and ensure Claude v2 is being called from Bedrock 
from langchain.llms.bedrock import Bedrock
from langchain import PromptTemplate

inference_modifier = {
    "max_tokens_to_sample": 4096,
    "temperature": 0.5,
    "top_k": 250,
    "top_p": 1,
    "stop_sequences": ["\n\nHuman"],
}

textgen_llm = Bedrock(
    model_id="anthropic.claude-v2:1",
    client=boto3_bedrock,
    model_kwargs=inference_modifier,
)

# **Examples:**

**Example 6.1 - Movie Review**

In the movie review prompt below, it's clear to a human reader that the second sentence belies the first. But Claude takes the word "unrelated" too literally.

In [4]:
response = textgen_llm("""

Human: Is this movie review sentiment positive or negative?

This movie blew my mind with its freshness and originality. Unrelatedly, I have been living under a rock since the year 1900.

Assistant:""")

print_ww(response)

 Based on the phrases "blew my mind" and "freshness and originality", the first sentence expresses a
very positive sentiment about the movie. However, the second sentence sarcastically suggests the
reviewer is out of touch, implying their praise may not be reliable. Overall I would say the review
leans positive but the second sentence introduces some skepticism or negativity. Without more
context, it's hard to determine an overall definitive sentiment.


**Example 6.2 - Better movie review**

To improve Claude's response, let's allow Claude to think things out first before answering. We do that by literally spelling out the steps that Claude should take in order to process and think through the prompt. 
Now, Claude's response is much better and more nuanced.

In [5]:
response = textgen_llm("""

Human: Is this review sentiment positive or negative? First write the best arguments for each side in <positive-arg> and <negative-arg> XML tags, then answer.

This movie blew my mind with its freshness and originality. Unrelatedly, I have been living under a rock since 1900.

Assistant:""")

print_ww(response)

 <positive-arg>
The review says the movie "blew my mind with its freshness and originality", which indicates a very
positive sentiment. This suggests the reviewer really enjoyed the originality and new ideas in the
movie.
</positive-arg>

<negative-arg>
The review sarcastically says the reviewer has been "living under a rock since 1900", implying they
are not familiar with movies and found this one fresh only because they have not seen many others.
This suggests the movie is not actually that original.
</negative-arg>

The positive argument is stronger here. The main statement expresses a clearly positive reaction to
the movie. The secondary statement about living under a rock is likely intended humorously rather
than a literal statement about the reviewer's movie knowledge. Overall this review has a positive
sentiment.


**Example 6.3 - Blockbusters**

Letting Claude think can shift Claude's answer from incorrect to correct. It's that simple in many cases where Claude makes mistakes!
Let's go through an example where Claude's answer is incorrect to see how asking Claude to think can fix that.	

In [6]:
response = textgen_llm("""

Human: Are both directors of Jaws and Casino Royale citizens of the same country?

Assistant:""")

print_ww(response)

 Unfortunately, I do not have enough context to definitively state whether the directors of Jaws and
Casino Royale are citizens of the same country. Jaws was directed by Steven Spielberg, an American
director. Casino Royale has had multiple directors over various film adaptations, and they may have
different citizenships. Without more details on which specific Casino Royale director you're
referring to, I cannot reliably determine if they share the same citizenship status as Spielberg.
I'd need the name of the specific Casino Royale director you're asking about to make that
comparison.


**Example 6.4 - Blockbusters 2**

Let's fix this by asking Claude to think step by step. Instead of spelling out each step for Claude, you can also simply use those exact words and ask Claude to think step by step first.			

In [13]:
response = textgen_llm("""

Human: Are both directors of Jaws and Casino Royale citizens of the same country? Make assumptions where needed, and think step by step.

Assistant:""")

print_ww(response)

 Okay, let's break this down step-by-step:

1) The director of Jaws is Steven Spielberg. As far as I can tell, Steven Spielberg is an American
citizen.

2) The director of Casino Royale is Martin Campbell. I wasn't able to definitively confirm his
citizenship, so I will make the assumption that Martin Campbell is a citizen of the United Kingdom.

3) The United States and the United Kingdom are different countries.

Therefore, based on the information I have and the assumptions made, the directors of Jaws (Steven
Spielberg) and Casino Royale (Martin Campbell) are citizens of different countries (United States
and United Kingdom respectively). The answer is no, they are not citizens of the same country.


**Example 6.5 - Simon says**

It's important to note that to some degree, "let's think step by step" has been pre-built into Claude's training. You will sometimes see Claude thinking step by step even when you don't ask it to, particularly if you asked it a logic or math problem. However, even then, such techniques are not failproof, and Claude thinks step by step and still gets the wrong answer. To help Claude here, we can tell Claude exactly what process it should undertake to think through the problem.

In [8]:
response = textgen_llm("""

Human: Use the following clues to answer the following multiple-choice question, using the following procedure:
(1) First, go through the clues one by one and consider whether the clue is potentially relevant
(2) Second, combine the relevant clues to reason out the answer to the question
(3) Third, map the answer to one of the multiple choice answers: either (a), (b), or (c)
 
Clues:
1. Miss Scarlett was the only person in the lounge.
2. The person with the pipe was in the kitchen.
3. Colonel Mustard was the only person in the observatory.
4. Professor Plum was not in the library nor the billiard room.
5. The person with the candlestick was in the observatory.
 
Question: Was Colonel Mustard in the observatory with the candlestick?
(a) Yes; Colonel Mustard was in the observatory with the candlestick
(b) No; Colonel Mustard was not in the observatory with the candlestick
(c) Unknown; there is not enough information to determine whether Colonel Mustard was in the observatory with the candlestick
 
Assistant:""")

print_ww(response)

 Okay, let's go through this step-by-step:

Relevant clues:
3. Colonel Mustard was the only person in the observatory.
5. The person with the candlestick was in the observatory.

Combining these clues, Colonel Mustard was the only person in the observatory, and the person with
the candlestick was also in the observatory. Since Colonel Mustard was the only person in the
observatory, he must have been the one with the candlestick.

Therefore, the answer is:
(a) Yes; Colonel Mustard was in the observatory with the candlestick

The key logical steps were:
1) Identify the relevant clues
2) Note that Colonel Mustard was alone in the observatory
3) The person with the candlestick was also in the observatory
4) Therefore, Colonel Mustard must have been the one with the candlestick


# **Exercises**

The following exercises will need you to manipulate the prompt to get the desired output

**Exercise 6.1 - Classifying Emails**

In this exercise, we'll be instructing Claude to sort emails into the following categories:
(A) Pre-sale question
(B) Broken or defective item
(C) Billing question
(D) Other (please explain)

For the first part of the exercise, change the prompt in the cell below to make Claude output the correct classification. Your answer needs to include the letter (A - D) of the correct choice, with the parentheses, as well as the name of the category.

> *Tip: Use precognition and other techniques you've learned leading up to this chapter!*

In [9]:
#Create a prompt template that has input variable(s)
multi_var_prompt = PromptTemplate(
    input_variables=["EMAIL"], #Change Input variables 
    template="""

Human: Please classify this email {EMAIL} 

Assistant:"""
)

# Pass in values to the input variable(s)
prompt = multi_var_prompt.format(EMAIL="Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.") #Should be classified as (B)

response = textgen_llm(prompt)

print_ww(response)

 Unfortunately I don't have enough context to definitively classify this email. However, based on
the description of the issue with the Mixmaster4000 making noise, smelling smoky, and seeming like
burning electronics, this could potentially be classified as:

- A product support request - The email describes a problem with a product (Mixmaster4000) and asks
for a replacement, indicating the need for product support.

- A warranty claim - If the product is still under warranty, the issue described would likely
qualify as a valid reason to request a replacement under the warranty coverage.

- A consumer complaint - The email has a negative tone regarding the apparent faulty product and
could be seen as a complaint about the quality or safety of the item.

Without knowing more details about the product, purchase timeline, expected resolution, etc., I can
only guess at categories based on the brief email content provided. Please let me know if you have
any other details that could help det

In [10]:
#Create a prompt template that has input variable(s)
multi_var_prompt = PromptTemplate(
    input_variables=["EMAIL"], #Change Input variables 
    template="""

Human: Please classify this email {EMAIL} 

Assistant:"""
)

# Pass in values to the input variable(s)
prompt = multi_var_prompt.format(EMAIL="Can I use my Mixmaster 4000 to mix paint, or is it only meant for mixing food?") #Should be classified as (A)

response = textgen_llm(prompt)

print_ww(response)

 Unfortunately I do not have enough context to definitively classify this email. However, based on
the content, it seems to be an informational inquiry about whether a particular appliance (the
Mixmaster 4000) can be used for mixing paint in addition to mixing food. Without more details or
background, I would categorize this as a general product usage question.


In [11]:
#Create a prompt template that has input variable(s)
multi_var_prompt = PromptTemplate(
    input_variables=["EMAIL"], #Change Input variables 
    template="""

Human: Please classify this email {EMAIL} 

Assistant:"""
)

# Pass in values to the input variable(s)
prompt = multi_var_prompt.format(EMAIL="I HAVE BEEN WAITING 4 MONTHS FOR MY MONTHLY CHARGES TO END AFTER CANCELLING!!  WHAT IS GOING ON???") #Should be classified as (C)

response = textgen_llm(prompt)

print_ww(response)

 Based on the content and tone, I would classify this as an angry customer complaint email. The use
of all caps and multiple punctuation indicates frustration. The customer seems to be complaining
that they have continued to be charged monthly fees despite having cancelled their service 4 months
ago. They are demanding an explanation for why the charges have persisted. The tone conveys anger
and urgency around getting this billing issue resolved.


In [12]:
#Create a prompt template that has input variable(s)
multi_var_prompt = PromptTemplate(
    input_variables=["EMAIL"], #Change Input variables 
    template="""

Human: Please classify this email {EMAIL} 

Assistant:"""
)

# Pass in values to the input variable(s)
prompt = multi_var_prompt.format(EMAIL="How did I get here I am not good with computer.  Halp.") #Should be classified as (D)

response = textgen_llm(prompt)

print_ww(response)

 Unfortunately I do not have enough context to definitively classify this email. However, based on
the content provided, a few possibilities come to mind:

- It could be someone asking for tech support or assistance using a computer, as they say "I am not
good with computer. Halp." The misspelling of "help" and the informal tone suggests this may be the
case.

- It could potentially be spam or some kind of phishing attempt, as the message seems vaguely
suspicious in its randomness and lack of context.

- Without more context, it's difficult to say conclusively. Additional details about the sender,
recipient, or circumstances around the email would help determine if it's a legitimate request for
help, a suspicious email, or something else. Based just on the content provided, those are some
possibilities that come to mind. Let me know if you have any other details that could help classify
the intent behind this email.


# Chapter 6 - END.